# Traveling Salesman Problem via Genetic algorithms

Francesco Andreuzzi -- Data Science and Scientific Computing

Final project for the course "Computational Physics Laboratory" (2022/2023)

## The problem

The Traveling Salesman Problem (TSP) is a well known problem in Computer Science. It consists in finding the optimal (i.e. cheapest) tour which visits all cities in the instance, given a matrix of costs to travel from one to another. 
$$
\normalsize
\begin{align*}
\text{Minimize}& \qquad \sum_{i=1}^N \sum_{j=1}^N x_{ij} d_{ij} \\
\text{with}& \qquad \sum_{j=1}^N x_{ij} = 1 \qquad i = 1, \dots N \quad (\text{Exactly one incoming})\\
& \qquad \sum_{i=1}^N x_{ij} = 1 \qquad j = 1 \dots N \quad (\text{Exactly one outgoing})
\end{align*}
$$

It's NP-hard, therefore exact algorithms are unfeasible for non-trivial instances.

## Genetic algorithms overview

Genetic algorithms (GA) are approximation randomized algorithms which iteratively optimize a partial set of viable solutions to a given problem, often called *chromosomes*. The optimization is carried out on a given *fitness* function, which in our case is the cost of a given tour.

Optimization happens by a peculiar process called *crossover*, which consists in merging two parent solutions into a new one, hopefully retained the best parts of the ancestors. Many *crossover operators* exist for the TSP problem, some of them are fully deterministic (e.g. `CX`, `CX2`), while others are based on a stochastic component (e.g. `OX`, `PMX`).

In addition to crossover, to further mimic natural biological evolution, GA leverage *mutation* to slightly modify a subset of exising chromosomes (choosed randomly). This often provides solutions not easily achievable by means of crossover, and is critical in fully exploring the solution manifold. Due to their intrinsic stochastic definition, mutation operator are always randomized procedures which operate on a single chromosome.

## Structure of the project

We briefly describe the structure of the GitHub repository.

### Languages

The problem was tackled using Python and Fortran 90, leveraging [f2py](https://numpy.org/doc/stable/f2py/index.html) to link the two languages. Python was used to develop the high level driver for the genetic framework, the problem loading and the data collection layer, and to perform final data analysis, due to its strong versatility. Fortran was used to develop the low-level mutation and crossover operators.

Thanks to the link between Python and Fortran it has been possible to test the fortran code by using the high level Python testing framework [pytest](https://pytest.org), which among standard testing traits provides many comfortable features like a rich CLI and [fixtures](https://docs.pytest.org/en/6.2.x/fixture.html).

### Directories

```diff
$ tree -I 'venv|__pycache__|*.mod' --dirsfirst
.
├── data
-   │   # Problem data in XML format
│   │   ...
├── data-loader
-   │   # Responsible for loading problem data from XML
│   ├── load.py
-   │   # Problem representation as a Python class
│   └── problem.py
├── run
-   │   # Argparse CLI + utilities (logging, text display)
│   ├── data_storage.py
│   ├── printer.py
│   ├── run.py
│   └── utils.py
├── tests
-   │   # Pytest testing for low-level Fortran routines
│   ├── crossover
│   │   ├── test_crossover.py
│   │   ├── test_cx2-original.py
│   │   ├── test_cx2.py
│   │   ├── test_cx.py
│   │   ├── test_ox.py
│   │   └── test_pmx.py
│   ├── mutation
│   │   ├── test_central_inverse.py
│   │   ├── test_mutation.py
│   │   ├── test_reverse_sequence.py
│   │   └── test_twors.py
│   ├── test_problem.py
│   └── test_utils.py
├── tsp-genetic-f90
-   │   # Fortran implementation of low-level operators
│   ├── crossover
│   │   ├── cx2.f90
│   │   ├── cx2-original.f90
│   │   ├── cx.f90
│   │   ├── ox.f90
│   │   └── pmx.f90
│   ├── mutation
│   │   ├── central_inverse.f90
│   │   ├── reverse_sequence.f90
│   │   └── twors.f90
│   ├── problem.f90
│   └── utils.f90
├── tsp-genetic-py
-   │   # Genetic framework implementation
│   ├── configuration.py
│   ├── crossover.py
│   ├── evolve.py
│   ├── idata_logger.py
│   └── mutation.py
├── LICENSE
├── Makefile
├── README.md
-   # We are here
├── tsp-genetic-algorithms.ipynb
-   # Fortran code compiled with f2py
└── tsp_genetic.cpython-39-x86_64-linux-gnu.so
```

In [3]:
import sys
import numpy as np
from pathlib import Path

sys.path.append("tsp-genetic-py")
from evolve import driver
from configuration import (
    Configuration,
    CrossoverStrategy,
    CrossoverRetainment,
    NextGenerationPolicy,
)
from crossover import Crossover
from mutation import Mutation

sys.path.append("run")
from printer import Printer
from data_storage import DataStorage

## Unexplored directions

It could be interesting to take into account various ways to generate the initial generation of chromosomes. In this project we've only considered the randomized permutation of the list of cities, but more refined approaches exist to guarantee a better coverage on the solution manifold.

## References

1. Deep, Kusum, and Hadush Mebrahtu. "New variations of order crossover for travelling salesman problem." International Journal of Combinatorial Optimization Problems and Informatics 2.1 (2011): 2-13.
1. Potvin, Jean-Yves. "Genetic algorithms for the traveling salesman problem." Annals of Operations Research 63 (1996): 337-370.
1. Hussain, Abid, et al. "Genetic algorithm for traveling salesman problem with modified cycle crossover operator." Computational intelligence and neuroscience 2017 (2017).
1. Kumar, Rakesh, Girdhar Gopal, and Rajesh Kumar. "Novel crossover operator for genetic algorithm for permutation problems." International Journal of Soft Computing and Engineering (IJSCE) 3.2 (2013): 252-258.
1. Abdoun, Otman, Jaafar Abouchabaka, and Chakir Tajani. "Analyzing the performance of mutation operators to solve the travelling salesman problem." arXiv preprint arXiv:1203.3099 (2012).
1. Hassanat, Ahmad, et al. "Choosing mutation and crossover ratios for genetic algorithms—a review with a new dynamic approach." Information 10.12 (2019): 390.
1. f2py official documentation: https://numpy.org/doc/stable/f2py/

## Open source software

- argparse: https://docs.python.org/3/library/argparse.html
- f2py: https://numpy.org/doc/stable/f2py/
- Matplotlib: https://matplotlib.org
- NumPy: https://numpy.org
- Python standard library: https://docs.python.org/3/library/
- pytest: https://pytest.org